###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es la Policía Nacional de Colombia

In [ ]:
import pandas as pd

###Archivo 37p5-impc.csv

Contiene información del delito de Terrorismo desde el 01 de enero del año 2010 al 30 de abril del año 2024.

In [ ]:
df = pd.read_csv("https://www.datos.gov.co/resource/37p5-impc.csv?$limit=50000")
df.head()

,departamento,municipio,codigo_dane,armas_medios,fecha_hecho,cantidad
0,PUTUMAYO,MOCOA (CT),86001000,ARTEFACTO EXPLOSIVO/CARGA DINAMITA,05/01/2010,1
1,ARAUCA,TAME,81794000,ARTEFACTO INCENDIARIO,06/01/2010,1
2,META,VILLAVICENCIO (CT),50001000,GRANADA DE MANO,06/01/2010,1
3,ARAUCA,FORTUL,81300000,CILINDRO BOMBA,07/01/2010,1
4,META,URIBE,50370000,MINA ANTIPERSONA,11/01/2010,1


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   departamento  4591 non-null   object
 1   municipio     4591 non-null   object
 2   codigo_dane   4591 non-null   int64 
 3   armas_medios  4591 non-null   object
 4   fecha_hecho   4591 non-null   object
 5   cantidad      4591 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 215.3+ KB


### Eliminación de columnas irrelevantes para el proyecto

In [ ]:
relevant_cols = ['codigo_dane', 'departamento', 'municipio', 'fecha_hecho', 'armas_medios', 'cantidad']
terrorism_crimes = df[relevant_cols]
terrorism_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   codigo_dane   4591 non-null   int64 
 1   departamento  4591 non-null   object
 2   municipio     4591 non-null   object
 3   fecha_hecho   4591 non-null   object
 4   armas_medios  4591 non-null   object
 5   cantidad      4591 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 215.3+ KB


### Convertir columna fecha_hecho a tipo date

In [ ]:
# Hacer una copia explícita del DataFrame
terrorism_crimes_ = terrorism_crimes.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime usando el formato correcto
terrorism_crimes_['fecha_hecho'] = pd.to_datetime(terrorism_crimes_['fecha_hecho'], format='%d/%m/%Y', errors='coerce', dayfirst=True)

# Extraer solo el año
terrorism_crimes_.loc[:, 'fecha_hecho'] = terrorism_crimes_['fecha_hecho'].dt.year

### Verificar valores nulos

In [ ]:
terrorism_crimes_.isnull().sum()

,0
codigo_dane,0
departamento,0
municipio,0
fecha_hecho,0
armas_medios,0
cantidad,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [ ]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['armas_medios', 'departamento', 'municipio']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(terrorism_crimes_[column].nunique())
    print()


Categorías en la columna 'armas_medios':
31

Categorías en la columna 'departamento':
29

Categorías en la columna 'municipio':
533



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [ ]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [ ]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    terrorism_crimes_[col] = terrorism_crimes_[col].apply(remove_accents_and_special_chars)

### Ajustar columna 'codigo_dane' para que coincida con el campo 'dept_mpio_code' de la tabla municipalities de la base de datos, que guarda toda la informacion de georeferenciacion de los municipios

 -  Cargar los datos con códigos reales de los municipios

Como producto de una consulta a la base de datos del proyecto que se esta construyendo (Tablas departments y municipalities) se creo el archivo csv que se carga en la siguiente celda, y que incluye los nombres de los departamentos y municipios con sus respectivos codigos, generados por el DANE (Estos codigos son los reales)

In [ ]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_dane" en el df terrorism_crimes_

In [ ]:
# Asegurarnos de que todos los valores en 'codigo_dane' sean strings
terrorism_crimes_['codigo_dane'] = terrorism_crimes_['codigo_dane'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = terrorism_crimes_['codigo_dane'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

False

In [ ]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [8 7]


In [ ]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,no_digitos_codigo_dane,count,percentage
0,8,3660,0.797212
1,7,931,0.202788


In [ ]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = terrorism_crimes_[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_dane']])
    print()

Muestra de registros con longitud 8:
  codigo_dane
0    86001000
1    81794000
2    50001000
3    81300000
4    50370000

Muestra de registros con longitud 7:
   codigo_dane
11     5001000
15     5847000
21     5315000
22     5154000
24     5001000



### Nota:

De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 79.7% tiene 8 dígitos (al parecer se le adicionaron 3 ceeros al final del código que realmente es de 5 dígitos), el 20,3% tiene solamente 7 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

  - Verificar si los codigo_dane de 7 dígitos corresponden a departamentos que se identifican con 1 dígito para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [ ]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtrado = terrorism_crimes_[terrorism_crimes_['codigo_dane'].str.len() == 7]

# Obtener las categorías únicas de la columna 'departamento'
categorias_departamento = filtrado['departamento'].unique()

# Imprimir las categorías
print(categorias_departamento)

['ANTIOQUIA' 'ATLANTICO']


Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane de un dígito, 5 y 8 respectivamente.

  - Adicionar un cero a los codigo_dane de 7 dígitos

In [ ]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_7(codigo):
    if len(codigo) == 7:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
terrorism_crimes_['codigo_dane'] = terrorism_crimes_['codigo_dane'].apply(add_zero_if_length_7)

In [ ]:
# Verificar cuantos codigo_dane de 7 dígitos quedaron
len(terrorism_crimes_[terrorism_crimes_['codigo_dane'].str.len() == 7])

0

  - Quitar los ultimos 3 ceros de los codigo_dane que tienen 8 dígitos

In [ ]:
# Función que corta los últimos 3 caracteres si la longitud del string es 8
def trim_last_3_if_length_8(codigo):
    if len(codigo) == 8:
        return codigo[:5]  # Dejar solo los primeros 5 caracteres
    return codigo

# Aplicar la función a la columna 'codigo'
terrorism_crimes_['codigo_dane'] = terrorism_crimes_['codigo_dane'].apply(trim_last_3_if_length_8)

In [ ]:
# Verificar cuantos codigo_dane de 8 dígitos quedaron
len(terrorism_crimes_[terrorism_crimes_['codigo_dane'].str.len() == 8])

0

- Comparar que los nombres de los departamentos en cada dataframe esten escritos correctamente

In [ ]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)
    # print(f"{label2} que no están en {label1}:")
    # print(only_in_list2)
    # print()


  - Comparar nombres de departamentos en los dataframes

In [ ]:
# Comparar listas de departamento
compare_lists(terrorism_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en terrorism_crimes_", "Departamentos dept_mpios_codes")

Departamentos en terrorism_crimes_ que no están en Departamentos dept_mpios_codes:
{'VALLE', 'GUAJIRA'}


  - Solucionar discrepancias en nombres de departamento

In [ ]:
# Diccionario de mapeo basado en los resultados de la comparación departamentos
depto_mapping = {'SAN ANDRES': 'ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA',
                 'VALLE': 'VALLE DEL CAUCA',
                 'GUAJIRA':'LA GUAJIRA'}

# Reemplazar los nombres incorrectos  el dataframe terrorism_crimes_
terrorism_crimes_['departamento'] = terrorism_crimes_['departamento'].replace(depto_mapping)

 - Confirmar que discrepancia se solucionó

In [ ]:
# Comparar listas de departamento
compare_lists(terrorism_crimes_['departamento'], dept_mpios_codes['dept_name'],
              "Departamentos en terrorism_crimes_", "Departamentos dept_mpios_codes")

Departamentos en terrorism_crimes_ que no están en Departamentos dept_mpios_codes:
set()


  - Verificar que todos los codigos esten estandarizados a 5 dígitos

In [ ]:
terrorism_crimes_['codigo_dane'].apply(len).nunique()

1

In [ ]:
terrorism_crimes_['codigo_dane'].apply(len).value_counts()

,count
codigo_dane,
5,4591


- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [ ]:
# Comparar listas de códigos
compare_lists(terrorism_crimes_['codigo_dane'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en terrorism_crimes_", "Códigos de municipios dept_mpios_codes")

Departamentos en terrorism_crimes_ que no están en Departamentos dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [ ]:
# Eliminar columnas innecesarias
columns_to_drop = ['departamento', 'municipio']
final_terrorism_crimes = terrorism_crimes_.drop(columns=columns_to_drop)

In [ ]:
# Adicionar columna para trazabilidad de la fuente
final_terrorism_crimes['source_id'] = 13

In [ ]:
final_terrorism_crimes.columns

Index(['codigo_dane', 'fecha_hecho', 'armas_medios', 'cantidad', 'source_id'], dtype='object')

In [ ]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fecha_hecho': 'year_of_incident',
    'cantidad': 'amount',
    'armas_medios': 'used_weapons',
    'codigo_dane': 'dane_code',
    'source_id': 'source_id'
}

# Renombrar las columnas
final_terrorism_crimes.rename(columns=translation_map, inplace=True)

In [ ]:
#Estructura final del dataset a integrar a la base de datos
final_terrorism_crimes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4591 entries, 0 to 4590
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   dane_code         4591 non-null   object
 1   year_of_incident  4591 non-null   int32 
 2   used_weapons      4591 non-null   object
 3   amount            4591 non-null   int64 
 4   source_id         4591 non-null   int64 
dtypes: int32(1), int64(2), object(2)
memory usage: 161.5+ KB


## Salvar en archivo csv en el drive

In [ ]:
# Guardar en archivos CSV en el drive
final_terrorism_crimes.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/terrorism_crimes.csv', index=False)